In [1]:
import numpy as np
from importlib import reload
import xmlTranslator as xmlP
from tqdm import tqdm
from utils import get_project_root

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
directory = '/data/iRealXML'
root = get_project_root()
myPath = str(root) + str(directory)

theChordDataset, theOffsetDataset, all_meta = xmlP.parse_info_from_XML(myPath)
print(theChordDataset.shape, theOffsetDataset.shape, len(all_meta))

100%|██████████| 4007/4007 [00:17<00:00, 226.30it/s]

(4006,) (4006,)
(4006,) (4006,) 4006


In [79]:
import importlib
from tqdm import tqdm
import xmlTranslator as xmlP
importlib.reload(xmlP)

id = 0
expand_form_dataset = []
for i in tqdm(range(theChordDataset.shape[0])):
    print(i)
    song_structure, status = xmlP.expand_song_structure(theChordDataset[i], i)
    if status == False:
        print('Song:', all_meta[i])
        id = i
        break
    expand_form_dataset.append(song_structure)

seq = theChordDataset[id].tolist()

print(seq)

  0%|          | 0/4006 [00:00<?, ?it/s]

 16%|█▌        | 646/4006 [00:00<00:01, 3320.42it/s]

0
Length of sequence: 137
first repeat done 27 0
next repeat: 33 1
second repeat done at stepper: 34 id: 0
first repeat done 101 2
next repeat: 111 3
second repeat done at stepper: 112 id: 2
Process completed successfully.. New form length: 181
-----------------------------

1
Length of sequence: 173
first repeat done 28 0
next repeat: 44 1
second repeat done at stepper: 45 id: 0
Process completed successfully.. New form length: 230
-----------------------------

2
Length of sequence: 202
first repeat done 99 0
next repeat: 105 1
second repeat done at stepper: 106 id: 0
Process completed successfully.. New form length: 244
-----------------------------

3
Length of sequence: 136
Process completed successfully.. New form length: 171
-----------------------------

4
Length of sequence: 182
first repeat done 28 0
next repeat: 45 1
second repeat done at stepper: 46 id: 0
first repeat done 102 2
next repeat: 116 3
second repeat done at stepper: 117 id: 2
Process completed successfully.. New

 36%|███▌      | 1443/4006 [00:00<00:00, 3788.15it/s]

Process completed successfully.. New form length: 164
-----------------------------

705
Length of sequence: 164
Process completed successfully.. New form length: 164
-----------------------------

706
Length of sequence: 186
Process completed successfully.. New form length: 218
-----------------------------

707
Length of sequence: 186
Process completed successfully.. New form length: 218
-----------------------------

708
Length of sequence: 188
first repeat done 52 0
next repeat: 61 1
second repeat done at stepper: 62 id: 0
first repeat done 154 2
next repeat: 163 3
second repeat done at stepper: 164 id: 2
Process completed successfully.. New form length: 237
-----------------------------

709
Length of sequence: 244
Process completed successfully.. New form length: 298
-----------------------------

710
Length of sequence: 113
Process completed successfully.. New form length: 168
-----------------------------

711
Length of sequence: 114
Process completed successfully.. New form le

...

first repeat done 151 0
next repeat: 169 1
first repeat done 169 1
Process completed successfully.. New form length: 310
-----------------------------

1591
Length of sequence: 185
Process completed successfully.. New form length: 313
-----------------------------

1592
Length of sequence: 203
first repeat done 85 0
next repeat: 108 1
second repeat done at stepper: 109 id: 0
Process completed successfully.. New form length: 295
-----------------------------

1593
No repetition data found
-----------------------------

1594
Length of sequence: 252
first repeat done 62 0
next repeat: 72 1
second repeat done at stepper: 73 id: 0
Process completed successfully.. New form length: 309
-----------------------------

1595
Length of sequence: 253
first repeat done 62 0
next repeat: 72 1
second repeat done at stepper: 73 id: 0
Process completed successfully.. New form length: 310
-----------------------------

1596
Length of sequence: 229
first repeat done 172 0
next repeat: 177 1
second repeat 

 76%|███████▌  | 3040/4006 [00:00<00:00, 3873.80it/s]

Length of sequence: 246
first repeat done 54 0
next repeat: 78 1
second repeat done at stepper: 79 id: 0
Process completed successfully.. New form length: 311
-----------------------------

2437
Length of sequence: 175
first repeat done 46 0
next repeat: 63 1
second repeat done at stepper: 64 id: 0
Process completed successfully.. New form length: 197
-----------------------------

2438
Length of sequence: 155
first repeat done 51 0
next repeat: 60 1
second repeat done at stepper: 61 id: 0
Process completed successfully.. New form length: 201
-----------------------------

2439
Length of sequence: 22
Process completed successfully.. New form length: 40
-----------------------------

2440
Length of sequence: 23
Process completed successfully.. New form length: 42
-----------------------------

2441
Length of sequence: 156
first repeat done 28 0
next repeat: 45 1
second repeat done at stepper: 46 id: 0
Process completed successfully.. New form length: 179
-----------------------------

2

100%|██████████| 4006/4006 [00:01<00:00, 3790.52it/s]

first repeat done 279 0
next repeat: 309 1
first repeat done 309 1
Process completed successfully.. New form length: 655
-----------------------------

3273
Length of sequence: 164
Process completed successfully.. New form length: 227
-----------------------------

3274
No repetition data found
-----------------------------

3275
No repetition data found
-----------------------------

3276
Length of sequence: 278
Process completed successfully.. New form length: 362
-----------------------------

3277
Length of sequence: 283
Process completed successfully.. New form length: 511
-----------------------------

3278
No repetition data found
-----------------------------

3279
Length of sequence: 111
Process completed successfully.. New form length: 133
-----------------------------

3280
Length of sequence: 90
first repeat done 25 0
next repeat: 40 1
second repeat done at stepper: 41 id: 0
Process completed successfully.. New form length: 110
-----------------------------

3281
Length of 

In [78]:
import importlib
importlib.reload(xmlP)
id = 287
print(all_meta[id])
song_structure, status = xmlP.expand_song_structure(theChordDataset[id], 41)

seq = theChordDataset[id].tolist()
print(seq)
print(song_structure)

{'composer': 'Chico Buarque', 'style': 'Samba', 'song_name': 'Apesar de Você  - In AB ABBB In 1', 'tonality': 'A major', 'midi_key': 21, 'time_signature': '4/4', 'decade': 'Null'}
Length of sequence: 269
first repeat done 28 0
next repeat: 39 1
second repeat done at stepper: 40 id: 0
first repeat done 247 2
next repeat: 257 3
first repeat done 257 3
All repetitions done
Process completed successfully.. New form length: 392
-----------------------------

['<style>', 'Samba', '|:', '.', 'A', '69', 'add 9 alter #5', '.', 'E', '7', 'add 9 alter #5', '|', '.', 'A', '69', 'add 9 alter #5', '.', 'E', '7', 'add 9 alter #5', '|', '.', 'A', '69', 'add 9 alter #5', '.', 'E', '7', 'add 9 alter #5', '|', 'Repeat_1', '.', 'A', '69', 'add 9 alter #5', '.', 'E', '7', 'add 9 alter #5', ':|', '|', 'Repeat_2', '.', 'A', '69', 'add 9', '.', 'D', '7', '.', 'C#', '7', '|', '.', 'F#', 'm7', '.', 'F', '|', '.', 'C#', 'm7', 'alter b5', '.', 'F#', '7', '|', '.', 'B', '9', '.', 'E', '7', '|', '.', 'A', '69', 'ad